In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

This notebook reads in and combines all 168 csvs into two dataframes and saves them as parquet files: 
1. first df contains the house_id (str), treated (bool), date_time (str), KWH/hh (float)
2. second df contains house_id (str), acorn (str), acorn_group (str)

In [2]:
usage_parts = []
acorn_parts = []
total_usage = pd.DataFrame()
total_acorn = pd.DataFrame()

In [3]:
for i in range(1, 169):
    file_name = 'data/London-Power-Networks/Power-Networks-LCL-June2015(withAcornGps)v2_{0}.csv'.format(i)
    usage_parts.append(pd.read_csv(file_name, usecols=[0, 1, 2, 3], header=0, names=['house_id', 'treated', 'date_time', 'KWH/hh'], 
                         na_values=['Null'], true_values=['ToU'], false_values=['Std']))
    acorn_parts.append(pd.read_csv(file_name, usecols=[0, 4, 5], header=0, names=['house_id', 'acorn_group', 'acorn_category']))
    

In [4]:
total_usage = pd.concat(usage_parts, ignore_index=True)
total_acorn = pd.concat(acorn_parts, ignore_index=True)

In [ ]:
total_acorn = total_acorn.drop_duplicates()
total_acorn.reset_index(drop=True, inplace=True)

In [5]:
# convert date_time from str to datetime
total_usage['date_time'] = total_usage['date_time'].astype('M8[ns]')

In [6]:
# taking out values that are not on the hh format
total_usage = total_usage[total_usage['date_time'].dt.minute % 30 == 0]

In [ ]:
# taking out datapoints that don't have a correct acorn category
total_acorn = total_acorn.loc[(total_acorn['acorn_category'] != 'ACORN-U') & 
                              (total_acorn['acorn_category'] != 'ACORN-')]

In [ ]:
# remove feb 29, 2012 from the 2012 data
total_usage_2012 = total_usage_2012.loc[(total_usage_2012['month'] != 2) & (total_usage_2012['day'] != 29)]

In [7]:
# let's seperate total_usage into difference years:
total_usage_2011 = total_usage.loc[total_usage['date_time'] < dt(2012, 1, 1)]
total_usage_2012 = total_usage.loc[(dt(2012, 1, 1) <= total_usage['date_time']) & (total_usage['date_time'] < dt(2013, 1, 1))]
total_usage_2013 = total_usage.loc[(dt(2013, 1, 1) <= total_usage['date_time']) & (total_usage['date_time'] < dt(2014, 1, 1))]
total_usage_2014 = total_usage.loc[dt(2014, 1, 1) <= total_usage['date_time']]

In [ ]:
# tariff mapper maps date: time to p/kWh
# only for 2013: jan through dec
tariffs = pd.read_csv('data/London-Power-Networks/tariffs_csv.csv', header=0, 
                      names=['date_time', 'tariff'])

# convert date_time from str to datetime
tariffs['date_time'] = tariffs['date_time'].astype('M8[ns]')

# A sub-set of 1,100 customers (Dynamic Time of Use or dToU) were given specific times when 
# their electricity tariff would be higher or lower price than normal – High (67.20p/kWh), 
# Low (3.99p/kWh) or normal (11.76p/kWh). The rest of the sample (around 4,500) were on a 
# flat rate of 14.228p/kWh.

tariffs['p/kWh'] = 0

# col[mask] = val

tariffs.loc[tariffs['tariff'] == 'High', 'p/kWh'] = 67.20
tariffs.loc[tariffs['tariff'] == 'Normal', 'p/kWh'] = 11.76
tariffs.loc[tariffs['tariff'] == 'Low', 'p/kWh'] = 3.99

tariffs = tariffs.set_index('date_time')

tariffs.head()

In [8]:
total_usage.to_parquet('total_usage.gzip', engine='auto', compression='gzip', index=False)
total_acorn.to_parquet('total_acorn.gzip', engine='auto', compression='gzip', index=False)

total_usage_2011.to_parquet('total_usage_2011.gzip', engine='auto', compression='gzip', index=False)
total_usage_2012.to_parquet('total_usage_2012.gzip', engine='auto', compression='gzip', index=False)
total_usage_2013.to_parquet('total_usage_2013.gzip', engine='auto', compression='gzip', index=False)
total_usage_2014.to_parquet('total_usage_2014.gzip', engine='auto', compression='gzip', index=False)

tariffs.to_parquet('tariffs.gzip', engine='auto', compression='gzip', index=True)